# Drive Downloader Notebook

Este cuaderno en Google Colab permite:
- Autenticar con tu cuenta de Google Workspace.
- Leer un CSV con columnas `version_id`, `version_title`, `ticket_id`, `ticket_title`, `ticket_description`, `ticket_user`, `url`.
- Crear una carpeta raíz de proyectos y carpetas para cada proyecto usando ticket_id - ticket_title.
- Crear accesos directos a los archivos y carpetas de Google Drive a esas carpetas disponibles en la columna url
- Registrar registros sin acceso en un fichero CSV ubicado en Drive


In [ ]:
# Instalar dependencias
!pip install --quiet --upgrade google-auth-oauthlib google-api-python-client

## Autenticar con usuario de Google

In [ ]:
# Autenticar con Google Workspace
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Crear cliente de Google Drive API
import google.auth
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds, _ = google.auth.default(scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=creds)
print('Autenticación completada.')

## 2. Funciones auxiliares

In [17]:
# Funciones auxiliares
import re
from urllib.parse import urlparse, parse_qs

from pathlib import Path
from googleapiclient.http import MediaIoBaseDownload
import io

INVALID_CHARS = r'[<>:\"/\\|?*]'

def sanitize_folder_name(name):
    return re.sub(INVALID_CHARS, '_', name)

def extract_drive_id(url: str) -> str:
    """
    Extrae el ID de Drive de URLs de distintos formatos, incluyendo:
    - https://drive.google.com/open?id=ID
    - https://drive.google.com/file/d/ID/view
    - https://drive.google.com/drive/folders/ID
    - https://docs.google.com/document/d/ID/edit
    """
    # 1) Intentar extraer “id” de la query (por ejemplo /open?id=…)
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    if 'id' in qs and qs['id']:
        return qs['id'][0]

    # 2) Limpiar parámetros (todo tras ‘?’) y buscar en el path
    clean = url.split('?', 1)[0]
    patterns = [
        r'/d/([A-Za-z0-9_-]{10,})',      # /file/d/ID
        r'folders/([A-Za-z0-9_-]{10,})', # /drive/folders/ID
        r'document/d/([A-Za-z0-9_-]{10,})' # /document/d/ID
    ]
    for pat in patterns:
        m = re.search(pat, clean)
        if m:
            return m.group(1)

    raise ValueError(f"No pude extraer Drive ID de: {url}")


## 3. Cargar el csv con las url de los recursos de cada proyecto

In [ ]:
# Subir y cargar el CSV de entrada
from google.colab import files
uploaded = files.upload()
# Asume que el CSV se llama 'versiones.csv'
csv_path = list(uploaded.keys())[0]
print(f"CSV cargado: {csv_path}")

## 4. Mapear drive con la carpeta /content/drive

El objetivo es que todo lo que se escriba en local se sincronice con Drive

In [ ]:
# Leer datos y preparar carpeta raíz
import pandas as pd
df = pd.read_csv(csv_path, encoding='utf-8-sig')

# 1) Monta tu Drive y vuelve a mapear “proyectos” dentro de él
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
# Ajusta la ruta bajo “Mi unidad” a la carpeta que quieres usar
# (debes tener un atajo en Mi unidad a la carpeta cuyo ID es 1AbCdEfGhIjKlMnOpQrStUvWxYz012)
root_dir = '/content/drive/Shareddrives/tu-carpeta'
os.makedirs(root_dir, exist_ok=True)

no_access = []
print('Datos cargados. Proyectos en:', root_dir)

## 5. Procesar el fichero csv [Ejecución]

Procesa el fichero csv
Crea las carpetas de proyecto
Crea los accesos directos de todos las url que forman parte de la documentación del proyecto

In [ ]:
import os
from googleapiclient.errors import HttpError

# Directorio raíz donde vive la carpeta de proyectos en Drive (montada o con SDK)
# si usas symlink, root_dir = '/content/proyectos'
# root_dir = '/content/proyectos'

# Ruta completa al fichero de log dentro de esa carpeta
log_path = os.path.join(root_dir, 'urls-sin-acceso.txt')

# Inicializa el log con cabecera
with open(log_path, 'w', encoding='utf-8') as f:
    f.write(','.join(df.columns.tolist()) + '\n')

# ID de la carpeta raíz en tu Drive donde van las carpetas de proyecto
ROOT_FOLDER_ID = '1eeK_v4NWU-1liJso2d-j8Yqi1Aw_7r8a'

# Guarda los folder_ids de proyecto para no recrearlos en cada iteración
project_folders = {}

# Recorre cada URL y crea un shortcut en Drive
for _, row in df.iterrows():
    proj_id    = row['ticket_id']
    proj_title = row['ticket_title']
    url        = row['url']

    # 1) Crear (o recuperar) carpeta de proyecto en Drive
    key = str(proj_id)
    if key not in project_folders:
        body = {
            'name'     : sanitize_folder_name(f"{proj_id} - {proj_title}"),
            'mimeType' : 'application/vnd.google-apps.folder',
            'parents'  : [ROOT_FOLDER_ID]
        }
        meta = drive_service.files().create(
            body=body,
            supportsAllDrives=True,
            fields='id'
        ).execute()
        project_folders[key] = meta['id']

    proj_folder_id = project_folders[key]

    # 2) Extraer el ID de Drive del URL
    try:
        target_id = extract_drive_id(url)
    except Exception as e:
        print(f"❌ No pude extraer ID de {url}: {e}")
        continue

    # 3) Leer el nombre real del target y usarlo
    try:
        target_meta = drive_service.files().get(
            fileId=target_id,
            fields='name',
            supportsAllDrives=True
        ).execute()
        target_name = target_meta.get('name', target_id)
    except HttpError as e:
        if e.status_code in (403, 404):
            print(f"❌ Sin acceso a metadata de {url}, lo registro y continúo.")
            with open(log_path, 'a', encoding='utf-8') as log:
                log.write(','.join(str(row[col]) for col in df.columns) + '\n')
            continue
        else:
            raise

    safe_name = sanitize_folder_name(target_name)

    shortcut_body = {
        'name'           : safe_name,
        'mimeType'       : 'application/vnd.google-apps.shortcut',
        'shortcutDetails': {'targetId': target_id},
        'parents'        : [proj_folder_id]
    }

    try:
        sc = drive_service.files().create(
            body=shortcut_body,
            supportsAllDrives=True,
            fields='id,name'
        ).execute()
        print(f"🔗 Shortcut creado: {sc['name']} → {url}")
    except HttpError as e:
        if e.status_code == 403:
            print(f"❌ Sin acceso creando shortcut para {url}, lo registro y continúo.")
            with open(log_path, 'a', encoding='utf-8') as log:
                log.write(','.join(str(row[col]) for col in df.columns) + '\n')
            continue
        else:
            raise



## 6. Dar acceso modo Lector - Funciones Auxiliares

Recorre todas las subcarpetas de documentación y le da acceso a una lista de
usuarios predefinida con rol Lector

In [12]:
from googleapiclient.errors import HttpError

def share_with_user(service, file_id: str, user_email: str):
    """
    Comprueba si user_email ya tiene permiso reader/writer/owner en file_id.
    Si no, crea permiso reader.
    """
    try:
        resp = service.permissions().list(
            fileId=file_id,
            fields='permissions(emailAddress,role)',
            supportsAllDrives=True
        ).execute()
        for p in resp.get('permissions', []):
            if p.get('emailAddress') == user_email and p.get('role') in ('reader','writer','owner'):
                print(f"🔓 Ya tenía permiso sobre {file_id} el usuario {user_email}")
                return
        service.permissions().create(
            fileId=file_id,
            body={'type':'user','role':'reader','emailAddress':user_email},
            supportsAllDrives=True,
            sendNotificationEmail=False
        ).execute()
        print(f"✅ Compartido {file_id} con {user_email}")
    except HttpError as e:
        print(f"⚠️ No pude compartir {file_id} con {user_email}: HTTP {e.resp.status}")

def share_shortcuts_in_subfolders(service, root_folder_id: str, user_emails: list[str]):
    """
    Recorre todas las subcarpetas inmediatas de root_folder_id (ordenadas ascendente por nombre),
    y para cada shortcut en ellas (no importa si apuntan a archivo o carpeta):
      - Si shortcut.targetMimeType es carpeta, lo omite.
      - Comparte el recurso de destino y el propio shortcut con user_emails.
    """
    # 1) Recuperar *todas* las subcarpetas inmediatas, no solo la primera página
    subfolders = []
    page_token = None
    while True:
        resp = service.files().list(
            q=f"'{root_folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false",
            spaces='drive',
            corpora='allDrives',
            supportsAllDrives=True,
            includeItemsFromAllDrives=True,
            fields='nextPageToken, files(id, name)',
            pageSize=500,            # opcional: hasta 1000
            pageToken=page_token
        ).execute()
        subfolders.extend(resp.get('files', []))
        page_token = resp.get('nextPageToken')
        if not page_token:
            break

    # 2) Ordenar ascendente por 'name' (case-insensitive)
    subfolders.sort(key=lambda f: f['name'].lower())

    # 3) Para cada subcarpeta, procesar sus shortcuts
    for sub in subfolders:
        folder_id = sub['id']
        folder_name = sub['name']
        print(f"▶ Procesando shortcuts en subcarpeta «{folder_name}» ({folder_id})")

        page_token = None
        while True:
            r = service.files().list(
                q=f"'{folder_id}' in parents and trashed=false",
                spaces='drive',
                corpora='allDrives',
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                fields='nextPageToken, files(id, name, mimeType, shortcutDetails)',
                pageToken=page_token
            ).execute()

            for f in r.get('files', []):
                if f['mimeType'] != 'application/vnd.google-apps.shortcut':
                    continue

                sid   = f['id']
                sname = f['name']
                sd    = f.get('shortcutDetails', {})
                target_id   = sd.get('targetId')
                target_mime = sd.get('targetMimeType')

                if not target_id:
                    print(f"⚠️ Atajo inválido sin target: {sname} ({sid})")
                    continue
                if target_mime == 'application/vnd.google-apps.folder':
                    print(f"📂 Compartiendo carpeta apuntada por atajo «{sname}» → {target_id}")
                    for user in user_emails:
                        # comparte la carpeta en sí, sin entrar a listar su contenido
                        share_with_user(service, target_id, user)
                    # no recursamos en su interior
                    continue

                print(f"🔗 Atajo «{sname}» → {target_id}")

                # Compartir destino y atajo
                for user in user_emails:
                    share_with_user(service, target_id, user)
                    share_with_user(service, sid,       user)

            page_token = r.get('nextPageToken')
            if not page_token:
                break


## 7. Dar acceso en Modo Lector - [ Ejecución ]

In [ ]:
# ─── Uso ───
# Define tu carpeta y lista de usuarios
ROOT_FOLDER_ID   = '1AbCdEfGhIjKlMnOpQrStUvWxYz012'
USERS       = ['direccion1@dominio.es', 'direccion2@dominio.es']

share_shortcuts_in_subfolders(drive_service, ROOT_FOLDER_ID, USERS)
